In [82]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix, hstack
train_df=pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,ID,Place,location,date,status,job_title,summary,positives,negatives,advice_to_mgmt,score_1,score_2,score_3,score_4,score_5,score_6,overall
0,1,startup_1,NaN,"Dec 11, 2018",Current Employee,Anonymous Employee,Best Company to work for,People are smart and friendly,Bureaucracy is slowing things down,NaN,4.0,5.0,5.0,4.0,5.0,0,5.0
1,2,startup_1,"Mountain View, CA","Jun 21, 2013",Former Employee,Program Manager,"Moving at the speed of light, burn out is inev...","1) Food, food, food. 15+ cafes on main campus ...",1) Work/life balance. What balance? All those ...,1) Don't dismiss emotional intelligence and ad...,2.0,3.0,3.0,5.0,3.0,2094,5.0
2,3,startup_1,"New York, NY","May 10, 2014",Current Employee,Software Engineer III,Great balance between big-company security and...,"* If you're a software engineer, you're among ...","* It *is* becoming larger, and with it comes g...",Keep the focus on the user. Everything else wi...,5.0,4.0,5.0,5.0,4.0,949,5.0
3,4,startup_1,"Mountain View, CA","Feb 8, 2015",Current Employee,Anonymous Employee,The best place I've worked and also the most d...,You can't find a more well-regarded company th...,I live in SF so the commute can take between 1...,Keep on NOT micromanaging - that is a huge ben...,2.0,5.0,5.0,4.0,5.0,498,4.0
4,10,startup_1,NaN,"Dec 9, 2018",Current Employee,Anonymous Employee,Execellent for engineers,Impact driven. Best tech in the world.,Size matters. Engineers are a bit disconnected...,NaN,5.0,5.0,5.0,5.0,5.0,0,4.0


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:

for col in train_df:
  dt=train_df[col].dtype
  if dt==float or dt==int:
      train_df[col].fillna(-999,inplace=True)
  else:
      train_df[col].replace(np.nan, "", regex=True,inplace=True)
      
for col in test_df:
  dt=test_df[col].dtype
  if dt==float or dt==int:
      test_df[col].fillna(-999,inplace=True)
  else:
      test_df[col].replace(np.nan, "", regex=True,inplace=True)

In [85]:
v = TfidfVectorizer()
v.fit(pd.concat([train_df.positives,test_df.positives]))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [86]:
positive_mat = v.transform(train_df['positives'])
negative_mat = v.transform(train_df['negatives'])
advice_to_mgmt_mat = v.transform(train_df['advice_to_mgmt'])
print(positive_mat.shape)
print(negative_mat.shape)
print(advice_to_mgmt_mat.shape)


mat=hstack([positive_mat,negative_mat,advice_to_mgmt_mat])
print(mat.shape)



(30336, 18935)
(30336, 18935)
(30336, 18935)
(30336, 56805)


In [87]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
clf = LogisticRegression()
X=mat
y=train_df.overall
clf.fit(X,y)
clf.score(X,y)
clf.predict(X)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [88]:

positive_mat = v.transform(test_df['positives'])

negative_mat = v.transform(test_df['negatives'])

advice_to_mgmt_mat = v.transform(test_df['advice_to_mgmt'])
print(positive_mat.shape)
print(negative_mat.shape)
print(advice_to_mgmt_mat.shape)


mat=hstack([positive_mat,negative_mat,advice_to_mgmt_mat])
print(mat.shape)

(29272, 18935)
(29272, 18935)
(29272, 18935)
(29272, 56805)


In [0]:
test_df['overall']=clf.predict(mat)

In [0]:
submission=test_df[['ID','overall']]
submission.to_csv('outputNLP_tfidf_cv.csv', index=False)
from google.colab import files
files.download('outputNLP_tfidf_cv.csv')